### 틀린 사용자가 유사한 문항 찾기(Item-based)

틀린 문제와 유사한 문항(틀린 사용자가 유사한 문제)을 추천할 때 활용

```python
import pandas as pd
df = pd.read_csv('datasets/skill_builder_data.csv', encoding = 'unicode_escape')
df.head()
```

In [1]:
import pandas as pd
df = pd.read_csv('./skill_builder_data.csv', encoding = 'unicode_escape')
df.head()

C:\Users\BIG3-09\AppData\Local\Temp\ipykernel_20240\1219256145.py:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./skill_builder_data.csv', encoding = 'unicode_escape')


,order_id,assignment_id,user_id,assistment_id,problem_id,original,correct,attempt_count,ms_first_response,tutor_mode,...,hint_count,hint_total,overlap_time,template_id,answer_id,answer_text,first_action,bottom_hint,opportunity,opportunity_original
0,33022537.0,277618,64525,33139,51424,1.0,1.0,1.0,32454.0,tutor,...,0.0,3.0,32454.0,30799.0,NaN,26,0.0,NaN,1.0,1.0
1,33022709.0,277618,64525,33150,51435,1.0,1.0,1.0,4922.0,tutor,...,0.0,3.0,4922.0,30799.0,NaN,55,0.0,NaN,2.0,2.0
2,35450204.0,220674,70363,33159,51444,1.0,0.0,2.0,25390.0,tutor,...,0.0,3.0,42000.0,30799.0,NaN,88,0.0,NaN,1.0,1.0
3,35450295.0,220674,70363,33110,51395,1.0,1.0,1.0,4859.0,tutor,...,0.0,3.0,4859.0,30059.0,NaN,41,0.0,NaN,2.0,2.0
4,35450311.0,220674,70363,33196,51481,1.0,0.0,14.0,19813.0,tutor,...,3.0,4.0,124564.0,30060.0,NaN,65,0.0,0.0,3.0,3.0


```python
problem_list = df['problem_id'].value_counts()[:314].index.tolist()
processed_df = df[df['problem_id'].isin(problem_list)]
```

In [2]:
df['problem_id'].value_counts()[:314]

problem_id
89817    757
91642    718
89270    698
89768    674
89290    643
        ... 
93237    204
93206    203
93152    202
93210    202
74282    200
Name: count, Length: 314, dtype: int64

In [3]:
problem_list = df['problem_id'].value_counts()[:314].index.tolist()
problem_list

[89817,
 91642,
 89270,
 89768,
 89290,
 89240,
 89799,
 89238,
 89809,
 91347,
 89274,
 89782,
 91661,
 89835,
 89769,
 89793,
 89820,
 91357,
 89849,
 89292,
 91396,
 89819,
 89877,
 91644,
 89818,
 91350,
 89284,
 91401,
 89848,
 89775,
 89275,
 89282,
 89844,
 91393,
 89271,
 91382,
 89281,
 91391,
 89276,
 91381,
 91400,
 89300,
 89279,
 91383,
 89278,
 91392,
 89805,
 89808,
 89789,
 89815,
 89273,
 189554,
 89876,
 89812,
 91376,
 89780,
 89272,
 91665,
 89241,
 89294,
 91646,
 89765,
 91365,
 89806,
 89779,
 91402,
 89285,
 89286,
 89293,
 91659,
 91353,
 89850,
 91641,
 89841,
 91667,
 91379,
 91658,
 91352,
 89243,
 91354,
 91363,
 91652,
 89761,
 89291,
 91662,
 89788,
 89770,
 89878,
 89834,
 93156,
 85650,
 89794,
 89767,
 89832,
 89871,
 89785,
 91660,
 89298,
 91638,
 53845,
 89280,
 91386,
 89843,
 89781,
 91664,
 89838,
 93150,
 89800,
 91398,
 84890,
 89725,
 85673,
 91655,
 89668,
 89879,
 89840,
 89762,
 91650,
 89704,
 89287,
 89297,
 91395,
 89265,
 89814,
 91388,

In [4]:
processed_df = df[df['problem_id'].isin(problem_list)]

```python
processed_df = processed_df[['user_id', 'problem_id', 'skill_name', 'correct']]
```

In [5]:
processed_df = processed_df[['user_id', 'problem_id', 'skill_name', 'correct']]

```python
processed_df = processed_df.dropna(subset=['skill_name'])
```

In [6]:
processed_df = processed_df.dropna(subset=['skill_name'])

```python
processed_df = processed_df.drop_duplicates(subset=['user_id', 'problem_id'])
```

In [7]:
processed_df = processed_df.drop_duplicates(subset=['user_id', 'problem_id'])

```python
processed_df['correct'] = processed_df['correct'].map({1.0 : 0.0, 0.0 : 1.0}) #유사한 학생들이 틀린 문제를 찾기 위해 0과 1을 바꿈
```

In [8]:
processed_df['correct'] = processed_df['correct'].map({1.0 : 0.0, 0.0 : 1.0}) #유사한 학생들이 틀린 문제를 찾기 위해 0과 1을 바꿈

```python
item_user_scoring = processed_df.pivot_table('correct', index = 'problem_id', columns = 'user_id')
```

In [9]:
item_user_scoring = processed_df.pivot_table('correct', index = 'problem_id', columns = 'user_id')

```python
item_user_scoring = item_user_scoring.fillna(0.0)
```

In [10]:
item_user_scoring = item_user_scoring.fillna(0.0)
item_user_scoring

user_id,21825,53167,64525,64634,70677,70679,70682,70684,70687,70688,...,96290,96291,96292,96293,96294,96295,96296,96297,96298,96299
problem_id,,,,,,,,,,,,,,,,,,,,,
49292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49307,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
189408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
189554,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


```python
from sklearn.metrics.pairwise import cosine_similarity

item_based_collabor = cosine_similarity(item_user_scoring)
```

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

item_based_collabor = cosine_similarity(item_user_scoring)

In [12]:
item_based_collabor

array([[1.        , 0.2057378 , 0.33681653, ..., 0.        , 0.        ,
        0.        ],
       [0.2057378 , 1.        , 0.16169042, ..., 0.        , 0.        ,
        0.        ],
       [0.33681653, 0.16169042, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

```python
item_based_collabor = pd.DataFrame(data = item_based_collabor, index = item_user_scoring.index, columns = item_user_scoring.index)
```

In [13]:
item_based_collabor = pd.DataFrame(data = item_based_collabor, index = item_user_scoring.index, columns = item_user_scoring.index)
item_based_collabor

problem_id,49292,49295,49301,49304,49307,49310,49313,49316,49319,49322,...,189217,189227,189239,189241,189360,189369,189408,189554,189565,189566
problem_id,,,,,,,,,,,,,,,,,,,,,
49292,1.000000,0.205738,0.336817,0.303895,0.243132,0.256776,0.283654,0.171184,0.187120,0.356348,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49295,0.205738,1.000000,0.161690,0.123091,0.218844,0.138675,0.131306,0.231125,0.202113,0.192450,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49301,0.336817,0.161690,1.000000,0.238833,0.318465,0.369970,0.286618,0.302703,0.235294,0.280056,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49304,0.303895,0.123091,0.238833,1.000000,0.193952,0.307255,0.290929,0.204837,0.358249,0.319801,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49307,0.243132,0.218844,0.318465,0.193952,1.000000,0.291343,0.275862,0.291343,0.127386,0.227429,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189369,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
189408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
189554,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


```python
prob_skill = df[['problem_id', 'skill_name']].drop_duplicates()

def get_similar_problem(item_id):
  #return item_based_collabor[item_id].sort_values(ascending=False)
  sim_df = pd.DataFrame(item_based_collabor[item_id].sort_values(ascending=False).reset_index())
  sim_df.columns = ['problem_id', 'similarity']
  sim_df = sim_df[sim_df['problem_id'] != item_id][:10]

  print("=========== ", item_id, "번 문항과 유사한 문제 TOP 10 (틀린 사용자가 유사한 문제)===========")
  skill_check = prob_skill[prob_skill['problem_id'] == item_id]
  for k in range(len(skill_check)):
    print(": ",skill_check['skill_name'].iloc[k], end = ' | ')
  print()

  for i in range(len(sim_df)):
    prob_id = sim_df['problem_id'].iloc[i]
    print("TOP ",i+1, "(",sim_df['similarity'].iloc[i].round(3),") : ", prob_id, "-", end = ' ')
    skill_check = prob_skill[prob_skill['problem_id'] == prob_id]
    for j in range(len(skill_check)):
      print(skill_check['skill_name'].iloc[j], end = ' | ')
    print()

```

In [14]:
prob_skill = df[['problem_id', 'skill_name']].drop_duplicates()

def get_similar_problem(item_id):
  #return item_based_collabor[item_id].sort_values(ascending=False)
  sim_df = pd.DataFrame(item_based_collabor[item_id].sort_values(ascending=False).reset_index())
  sim_df.columns = ['problem_id', 'similarity']
  sim_df = sim_df[sim_df['problem_id'] != item_id][:10]

  print("=========== ", item_id, "번 문항과 유사한 문제 TOP 10 (틀린 사용자가 유사한 문제)===========")
  skill_check = prob_skill[prob_skill['problem_id'] == item_id]
  for k in range(len(skill_check)):
    print(": ",skill_check['skill_name'].iloc[k], end = ' | ')
  print()

  for i in range(len(sim_df)):
    prob_id = sim_df['problem_id'].iloc[i]
    print("TOP ",i+1, "(",sim_df['similarity'].iloc[i].round(3),") : ", prob_id, "-", end = ' ')
    skill_check = prob_skill[prob_skill['problem_id'] == prob_id]
    for j in range(len(skill_check)):
      print(skill_check['skill_name'].iloc[j], end = ' | ')
    print()

```python
get_similar_problem(49346)
```

In [15]:
get_similar_problem(49346)

===========  49346 번 문항과 유사한 문제 TOP 10 (틀린 사용자가 유사한 문제)===========
:  Subtraction Whole Numbers | :  Pattern Finding  | 
TOP  1 ( 0.457 ) :  49325 - Subtraction Whole Numbers | Pattern Finding  | 
TOP  2 ( 0.391 ) :  49343 - Subtraction Whole Numbers | Pattern Finding  | 
TOP  3 ( 0.348 ) :  49337 - Subtraction Whole Numbers | Pattern Finding  | 
TOP  4 ( 0.348 ) :  49313 - Subtraction Whole Numbers | Pattern Finding  | 
TOP  5 ( 0.327 ) :  49304 - Subtraction Whole Numbers | Pattern Finding  | 
TOP  6 ( 0.321 ) :  49373 - Subtraction Whole Numbers | Pattern Finding  | 
TOP  7 ( 0.311 ) :  49358 - Subtraction Whole Numbers | Pattern Finding  | 
TOP  8 ( 0.304 ) :  49352 - Subtraction Whole Numbers | Pattern Finding  | 
TOP  9 ( 0.295 ) :  89295 - Area Parallelogram | 
TOP  10 ( 0.286 ) :  49310 - Subtraction Whole Numbers | Pattern Finding  | 


```python
prob_list = item_based_collabor.index.tolist()

for prob in prob_list:
  get_similar_problem(prob)
```


In [16]:
prob_list = item_based_collabor.index.tolist()

for prob in prob_list:
  get_similar_problem(prob)

===========  49292 번 문항과 유사한 문제 TOP 10 (틀린 사용자가 유사한 문제)===========
:  Subtraction Whole Numbers | :  Pattern Finding  | 
TOP  1 ( 0.378 ) :  49373 - Subtraction Whole Numbers | Pattern Finding  | 
TOP  2 ( 0.372 ) :  49358 - Subtraction Whole Numbers | Pattern Finding  | 
TOP  3 ( 0.356 ) :  49322 - Subtraction Whole Numbers | Pattern Finding  | 
TOP  4 ( 0.337 ) :  49301 - Subtraction Whole Numbers | Pattern Finding  | 
TOP  5 ( 0.319 ) :  49364 - Subtraction Whole Numbers | Pattern Finding  | 
TOP  6 ( 0.304 ) :  49304 - Subtraction Whole Numbers | Pattern Finding  | 
TOP  7 ( 0.284 ) :  49313 - Subtraction Whole Numbers | Pattern Finding  | 
TOP  8 ( 0.274 ) :  49331 - Subtraction Whole Numbers | Pattern Finding  | 
TOP  9 ( 0.273 ) :  49343 - Subtraction Whole Numbers | Pattern Finding  | 
TOP  10 ( 0.273 ) :  49346 - Subtraction Whole Numbers | Pattern Finding  | 
===========  49295 번 문항과 유사한 문제 TOP 10 (틀린 사용자가 유사한 문제)===========
:  Subtraction Whole Numbers | :  Pattern Finding  

### TODO : 유사한 문제를 틀린 학생 찾기 (User-based)

hint : pivot_table의 인덱스가 problem_id 라면 문항간 유사도를 구할 수 있고, pivot_table의 인덱스가 user_id가 되면, 사용자간 유사도를 구할 수 있다.

In [22]:
processed_df.pivot_table('correct', index = 'problem_id', columns = 'user_id').fillna(0)

user_id,21825,53167,64525,64634,70677,70679,70682,70684,70687,70688,...,96290,96291,96292,96293,96294,96295,96296,96297,96298,96299
problem_id,,,,,,,,,,,,,,,,,,,,,
49292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49307,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
189408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
189554,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
user_item_scoring = processed_df.pivot_table('correct', index = 'user_id', columns = 'problem_id').fillna(0)
user_item_scoring

problem_id,49292,49295,49301,49304,49307,49310,49313,49316,49319,49322,...,189217,189227,189239,189241,189360,189369,189408,189554,189565,189566
user_id,,,,,,,,,,,,,,,,,,,,,
21825,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64525,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70677,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96297,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
user_based_collabor = cosine_similarity(user_item_scoring)

In [20]:
item_based_collabor = pd.DataFrame(data = item_based_collabor, index = item_user_scoring.index, columns = item_user_scoring.index)
item_based_collabor

problem_id,49292,49295,49301,49304,49307,49310,49313,49316,49319,49322,...,189217,189227,189239,189241,189360,189369,189408,189554,189565,189566
problem_id,,,,,,,,,,,,,,,,,,,,,
49292,1.000000,0.205738,0.336817,0.303895,0.243132,0.256776,0.283654,0.171184,0.187120,0.356348,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49295,0.205738,1.000000,0.161690,0.123091,0.218844,0.138675,0.131306,0.231125,0.202113,0.192450,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49301,0.336817,0.161690,1.000000,0.238833,0.318465,0.369970,0.286618,0.302703,0.235294,0.280056,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49304,0.303895,0.123091,0.238833,1.000000,0.193952,0.307255,0.290929,0.204837,0.358249,0.319801,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49307,0.243132,0.218844,0.318465,0.193952,1.000000,0.291343,0.275862,0.291343,0.127386,0.227429,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189369,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
189408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
189554,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### TODO : 밀크T 문항별 정오답 데이터 사용

In [52]:
df = pd.read_csv('./초등_4학년_수학_차시문항풀이이력.csv',index_col=0)
df.head()

,사용자ID,차시코드,단원코드,단원제목,강의내용,강의명,학년,학기,시험구분,강의구분,...,대단원코드,대단원제목,중단원코드,중단원제목,소단원코드,소단원제목,토픽코드,토픽제목,난이도,평가영역
0,62fbd7b7-6a32-4da2-b203-acdec95e00b2,T0ME41U43001,T0ME41U43,1학기 복습,"만, 다섯 자리 수 알아보기",수학 1단원 【복습①】,4,1,NaN,E,...,17120995.0,1. 큰 수,14201237.0,다섯 자리 수,12233514.0,다섯 자리 수의 이해,12234054.0,모형 돈 세어 보기,2.0,91.0
1,62fbd7b7-6a32-4da2-b203-acdec95e00b2,T0ME41U43001,T0ME41U43,1학기 복습,"만, 다섯 자리 수 알아보기",수학 1단원 【복습①】,4,1,NaN,E,...,17120995.0,1. 큰 수,14201237.0,다섯 자리 수,12233514.0,다섯 자리 수의 이해,12233718.0,10000 알아보기,2.0,91.0
2,62fbd7b7-6a32-4da2-b203-acdec95e00b2,T0ME41U43001,T0ME41U43,1학기 복습,"만, 다섯 자리 수 알아보기",수학 1단원 【복습①】,4,1,NaN,E,...,17120995.0,1. 큰 수,14201237.0,다섯 자리 수,12233514.0,다섯 자리 수의 이해,12234058.0,"10000이 ■, 1000이 ▲ ……인 다섯 자리 수를 나타내기",2.0,91.0
3,62fbd7b7-6a32-4da2-b203-acdec95e00b2,T0ME41U43001,T0ME41U43,1학기 복습,"만, 다섯 자리 수 알아보기",수학 1단원 【복습①】,4,1,NaN,E,...,17120995.0,1. 큰 수,14201237.0,다섯 자리 수,12233515.0,다섯 자리 수의 자릿값,12233721.0,각 자릿수의 합으로 나타내기,3.0,91.0
4,62fbd7b7-6a32-4da2-b203-acdec95e00b2,T0ME41U43001,T0ME41U43,1학기 복습,"만, 다섯 자리 수 알아보기",수학 1단원 【복습①】,4,1,NaN,E,...,17120995.0,1. 큰 수,14201237.0,다섯 자리 수,12233515.0,다섯 자리 수의 자릿값,12233719.0,각 자리의 숫자와 나타내는 값 알아보기,3.0,91.0


In [53]:
#user : 사용자ID
#item : 강의명
#values : 학년, 학기, 강의구분, 난이도, 평가영역
#추천모델에 쓸 내용 : 정오답

In [54]:
print(df['정오답'])
print(df['정오답'].value_counts()) 

0        O
1        O
2        X
3        O
4        O
        ..
99995    O
99996    O
99997    O
99998    O
99999    O
Name: 정오답, Length: 100000, dtype: object
정오답
O    65116
X    27009
Name: count, dtype: int64


In [55]:
#정오답 변경
df['정오답'] = df['정오답'].map({'O':1.0,'X':0.0})

In [62]:
user = df.pivot_table('정오답',index='사용자ID',columns='강의명')

In [63]:
user = user.fillna(0.0)

In [69]:
#print(cosine_similarity(user))
user_col = pd.DataFrame(cosine_similarity(user),index=user.index,columns=user.index)

In [76]:
from sklearn.neighbors import NearestNeighbors
nei = NearestNeighbors(n_neighbors=10,metric='cosine').fit(np.array(user.values))
import numpy as np
neighbor = nei.kneighbors()

In [78]:
print(neighbor[1]) #가장 유사한 것 상위 10개 표시, id - index mapping하면 검색 가능

[[35 18 30 19 27  7 39 12 21  1]
 [37 36 34 17 39 35  0 24 10 33]
 [ 6 20 27 12 29 19  9 41 44 30]
 [14  0 35  7 18 30 19 27 21 39]
 [10 43 34 17 36 32 44 22 15  9]
 [19 27 35  0 30 18 20 38  7 39]
 [ 2 20 41 36 44  9 29 27 12 32]
 [35 18  0 30 27 19 39 12 20 41]
 [34 36 25 17 44  1 39  9 10 27]
 [27 44 39 24  1  8 20  0 10 17]
 [43 34 44 28 17 36 15 32  1 37]
 [35  0 19 30 18 21 27 31  7 38]
 [20 27  0 35 18  7 30 41 19 16]
 [ 7 30 35 18  0 19 39 27 33  8]
 [ 3  1 39 34 44 10  8 24 36 33]
 [10 28 43 22  1 34 17 36 44 37]
 [18  0 27 41 35 30 12  1  7 17]
 [36 10  1 34 37 44  8 39 27 16]
 [35  0 30  7 27 19 39 12 21 41]
 [ 0 30 35 27 18  7 39 12 24 37]
 [12 41 27  7  0 35 18 30 19 16]
 [ 0 35 30 18  7 27 19 38 39 16]
 [15 43 10 28 44  1 12 24 37 34]
 [21 11 35 30  0 19 38 18  7 13]
 [39  1 19  9 27 37  0 33 36 35]
 [34  8 36 44 32 10 17  9 33  1]
 [40 11 42 41  0 38 20 35 27 13]
 [ 0 35 30 19 18  7 12 39 20 41]
 [43 10 15 32 36 22 44 34  1 17]
 [20 36  7 12 17 27  5 34 18  8]
 [ 0 35 18

In [79]:
item = df.pivot_table('정오답',index='강의명',columns='사용자ID')

In [81]:
item = item.fillna(0.0)

In [84]:
cos = cosine_similarity(item)

In [85]:
item_col = pd.DataFrame(cos,index=item.index,columns=item.index)

In [87]:
nei = NearestNeighbors(n_neighbors=10,metric='cosine').fit(item_col)

In [90]:
neighbors = nei.kneighbors()
print(neighbors[1]) #비슷한 문제 상위 10개 metric cosine 

[[ 83  27  59 ...  52 119 118]
 [ 26 125 126 ... 101  53  54]
 [ 86   6 108 ... 110   3 111]
 ...
 [145 101 100 ... 147  49 103]
 [144 104 149 ... 101 103 147]
 [  0  27  83 ... 113   9  37]]


Appendix) 행동데이터를 기반한 것 이므로 항상 같은 단원, 같은 개념을 가지는 문항들이 높은 유사도를 가지는 것은 아님.

> Prob 1  | x x x 0 1 1 0 0 x x x

> Prob 2  | x x x 1 1 0 0 1 x x x

> Prob 3  | 0 0 1 x x x x x 1 0 1

- Prob1 과 Prob3 이 같은 개념에 관한 문제더라도, 풀이한 학생이 다르면 유사도를 가지지 못함
- Prob1 과 Prob2 가 다른 개념이더라도, 심지어 정오답 이력이 비슷하지 않더라도, 풀이한 학생들이 같다면 같은 개념인 Prob3보다 높은 유사도를 가지게 됨
